# EDA Données changement climatique - LSH (Longues Séries Homogénéisées)
Séries mensuelles corrigées par homogénéisation statistique pour les paramètres températures minimales (TN), maximales (TX), insolations (IN) ou précipitations (RR).

Liste des fichier qui vont être étudiés : 
- SH_IN pas de lien pour l'outremer correspond au cumul mensuel des durées d'insolation. 
- SH_RR_Outremer ( cumul mensuel des hauteurs de précipitation en mm et 1/10)
- SH_TN_Outremer (Moyenne mensuelle de la température minimal )
- SH_TX_Outremer (Moyenne mensuelle de la température maximal sous abri)


In [42]:
import requests 
import zipfile
import io
import pandas as pd
import os



In [43]:

# Lire depuis un zip et combiner

def extract_csv_to_df(urls):
    response = requests.get(urls)
        
    if response.status_code == 200:
        array_header = []
        array_data = []
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
            for filename in zip_file.namelist():
                if filename.endswith('.csv'):
                    # on sépare la lecture des métadonnées et des données
                    print(f"Lecture de {filename}...")
                    with zip_file.open(filename) as file:                
                        df_header = pd.read_csv(file).head(12)
                    
                    with zip_file.open(filename) as file:
                        df_data = pd.read_csv(file, sep=';', skiprows=13)
                        
                        # On récupère le num_post
                        num_post = df_header.iloc[0,0].split('=')[1].strip()
                        # Ajouter une colonne num_postpour tracer l'origine des mesures
                        df_data.insert(0, 'num_post', num_post)
                        array_header.append(df_header)
                        array_data.append(df_data)
        return array_header, array_data
        



In [44]:
array_header, array_data = extract_csv_to_df("https://www.data.gouv.fr/api/1/datasets/r/9617eade-a4ae-4fa1-bbe7-88458aff67b2")


Lecture de SH_MRR097105002.csv...
Lecture de SH_MRR097107002.csv...
Lecture de SH_MRR097108003.csv...
Lecture de SH_MRR097111001.csv...
Lecture de SH_MRR097115003.csv...
Lecture de SH_MRR097117002.csv...
Lecture de SH_MRR097118001.csv...
Lecture de SH_MRR097124004.csv...
Lecture de SH_MRR097128008.csv...
Lecture de SH_MRR097209017.csv...
Lecture de SH_MRR097304001.csv...
Lecture de SH_MRR097305001.csv...
Lecture de SH_MRR097409230.csv...
Lecture de SH_MRR097410238.csv...
Lecture de SH_MRR097411150.csv...
Lecture de SH_MRR097418110.csv...
Lecture de SH_MRR098715003.csv...
Lecture de SH_MRR098718001.csv...
Lecture de SH_MRR098722003.csv...
Lecture de SH_MRR098722005.csv...
Lecture de SH_MRR098723001.csv...
Lecture de SH_MRR098723003.csv...
Lecture de SH_MRR098723004.csv...
Lecture de SH_MRR098731002.csv...
Lecture de SH_MRR098738010.csv...
Lecture de SH_MRR098747002.csv...
Lecture de SH_MRR098747013.csv...
Lecture de SH_MRR098748003.csv...
Lecture de SH_MRR098748009.csv...
Lecture de SH_

In [45]:
def format_metadata(df_meta):
    lines = df_meta.iloc[:, 0].tolist() # extrait la colonne sous forme de liste
    data = {}
    for line in lines:
        line = line.strip()
        if not line.startswith("#"):
            continue
        # enlève le "#"
        content = line[1:].strip()
        # découpe en clé=valeur
        if "=" in content:
            key, value = content.split("=", 1)
            key = key.strip()
            value = value.strip()
            data[key] = value
        elif ":" in content:
            key, value = content.split(":", 1)
            key = key.strip()
            value = value.strip()
            data[key] = value
    # transforme en DataFrame avec 1 seule ligne
    df_meta = pd.DataFrame([data])
    print(df_meta)
    return df_meta

In [46]:
format_metadata(array_header[0]).head()

  NUM_POSTE             NOM_USUEL LATITUDE (°) LONGITUDE(°) ALTITUDE (m)  \
0  97105002  BASSE-TERRE GUILLARD       15.996      -61.721           92   

  ETAT DU POSTE (fermé avec date de fermeture AAAA-MM-JJ ou ouvert à la date de production du fichier le 13/03/2025)  \
0                                             ouvert                                                                   

      NUM_SERIE Amplitude minimale détectable de la série  \
0  MRR097105002                                      0.12   

  Période homogénéisée Date(s) de rupture(s) d'homogénéité  \
0        196501 201612                              197512   

  Periode homogénéisée  196501 201612  : Q_HOM  \
0                                            1   

  Données prolongées si nécessaire jusqu'en 202412 par des donnees mensuelles non homogénéisées : Q_HOM  
0                                                  0                                                     


,NUM_POSTE,NOM_USUEL,LATITUDE (°),LONGITUDE(°),ALTITUDE (m),ETAT DU POSTE (fermé avec date de fermeture AAAA-MM-JJ ou ouvert à la date de production du fichier le 13/03/2025),NUM_SERIE,Amplitude minimale détectable de la série,Période homogénéisée,Date(s) de rupture(s) d'homogénéité,Periode homogénéisée 196501 201612 : Q_HOM,Données prolongées si nécessaire jusqu'en 202412 par des donnees mensuelles non homogénéisées : Q_HOM
0,97105002,BASSE-TERRE GUILLARD,15.996,-61.721,92,ouvert,MRR097105002,0.12,196501 201612,197512,1,0


In [47]:
# Combiner tous les DataFrames
df_combined = pd.concat(array_data, ignore_index=True)

In [48]:

# dataframes to csv
def df_to_csv(df, output_file):
    os.makedirs(os.path.dirname(output_file), exist_ok=True) # Créer le répertoire s'il n'existe pas
    
    # Sauvegarder
    df.to_csv(output_file, index=False)
    print(f"✓ Sauvegardé dans {output_file}")
        
    

In [49]:
def format_stackv_metadata(array_df_metadata):
    frames = []
    for df_meta in array_df_metadata:
        frames.append(format_metadata(df_meta))
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

In [50]:
def split_and_combine_csv(key, url):
    array_header, array_data = extract_csv_to_df(url)
    df_metadata = format_stackv_metadata(array_header)
    df_combined = pd.concat(array_data, ignore_index=True)
    df_data = df_combined.rename(columns={"VALEUR": key})
    return df_metadata, df_data


In [51]:
urls = {
    'Precipîtations': "https://www.data.gouv.fr/api/1/datasets/r/9617eade-a4ae-4fa1-bbe7-88458aff67b2",
    'Max_Temp': "https://www.data.gouv.fr/api/1/datasets/r/69f2f61e-781c-4132-84c1-dee3e0becbbb",
    'Min_Temp': "https://www.data.gouv.fr/api/1/datasets/r/9cfc5c85-3a23-4a2b-99b6-57ae76ce25b4"
}

In [52]:
for key, url in urls.items():
    df_metadata, df_data = split_and_combine_csv(key, url)
    df_to_csv(df_metadata, output_file=f"../data2/{key.lower()}_metadata.csv")
    df_to_csv(df_data, output_file=f"../data2/{key.lower()}_data.csv")

Lecture de SH_MRR097105002.csv...
Lecture de SH_MRR097107002.csv...
Lecture de SH_MRR097108003.csv...
Lecture de SH_MRR097111001.csv...
Lecture de SH_MRR097115003.csv...
Lecture de SH_MRR097117002.csv...
Lecture de SH_MRR097118001.csv...
Lecture de SH_MRR097124004.csv...
Lecture de SH_MRR097128008.csv...
Lecture de SH_MRR097209017.csv...
Lecture de SH_MRR097304001.csv...
Lecture de SH_MRR097305001.csv...
Lecture de SH_MRR097409230.csv...
Lecture de SH_MRR097410238.csv...
Lecture de SH_MRR097411150.csv...
Lecture de SH_MRR097418110.csv...
Lecture de SH_MRR098715003.csv...
Lecture de SH_MRR098718001.csv...
Lecture de SH_MRR098722003.csv...
Lecture de SH_MRR098722005.csv...
Lecture de SH_MRR098723001.csv...
Lecture de SH_MRR098723003.csv...
Lecture de SH_MRR098723004.csv...
Lecture de SH_MRR098731002.csv...
Lecture de SH_MRR098738010.csv...
Lecture de SH_MRR098747002.csv...
Lecture de SH_MRR098747013.csv...
Lecture de SH_MRR098748003.csv...
Lecture de SH_MRR098748009.csv...
Lecture de SH_